In [1]:
# ATTENTION: Please do not alter any of the provided code in the exercise. Only add your own code where indicated
# ATTENTION: Please do not add or remove any cells in the exercise. The grader will check specific cells based on the cell position.
# ATTENTION: Please use the provided epoch values when training.

import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd

In [2]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    images = []
    labels = []
    with open(filename) as training_file:
      # Your code starts here
        lines = training_file.readlines()
        for line in lines[1:]:
            row = line.strip().split(',')
            labels.append(float(row[0]))
            images.append([float(x) for x in row[1:785]])
        images = np.array([np.array_split(r, 28) for r in images])
        labels = np.array(labels)
      # Your code ends here
    return images, labels

path_sign_mnist_train = f"{getcwd()}/../tmp2/sign_mnist_train.csv"
path_sign_mnist_test = f"{getcwd()}/../tmp2/sign_mnist_test.csv"
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [3]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = np.expand_dims(training_images, axis=-1)
testing_images = np.expand_dims(testing_images, axis=-1)


# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=1,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      zoom_range=0.01,
      #horizontal_flip=True,
      )

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

   
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [4]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
n_classes=25


train_generator = train_datagen.flow(training_images, training_labels, batch_size=85)
validation_generator = validation_datagen.flow(testing_images, testing_labels)

import tensorflow as tf
import tensorflow.keras.layers as L
model = tf.keras.models.Sequential([
    # Your Code Here
    L.Conv2D(16, (3,3), padding='SAME', input_shape=[28,28,1], activation='relu'),
    L.MaxPooling2D(pool_size=(2,2)),
    L.Conv2D(32, (3,3), padding='SAME', activation='relu'),
    L.MaxPooling2D(pool_size=(2,2)),
    L.Flatten(),
    L.Dense(n_classes, activation='softmax')
])

# Compile Model. 
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

# Train the Model
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch = training_images.shape[0] // 85,
                              epochs = 3,
                              validation_steps = training_images.shape[0] // 85,
)

model.evaluate(testing_images, testing_labels, verbose=0)

Epoch 1/3
323/323 [==============================] - 47s 146ms/step - loss: 1.5697 - acc: 0.5642 - val_loss: 0.8272 - val_acc: 0.7509
Epoch 2/3
323/323 [==============================] - 40s 124ms/step - loss: 0.3868 - acc: 0.8814 - val_loss: 0.5802 - val_acc: 0.8185
Epoch 3/3
323/323 [==============================] - 38s 118ms/step - loss: 0.1483 - acc: 0.9570 - val_loss: 0.5259 - val_acc: 0.8537


[185.30181163951664, 0.69241494]

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Submission Instructions

In [ ]:
# Now click the 'Submit Assignment' button above.

# When you're done or would like to take a break, please run the two cells below to save your work and close the Notebook. This will free up resources for your fellow learners. 

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
IPython.notebook.session.delete();
window.onbeforeunload = null
setTimeout(function() { window.close(); }, 1000);